In [171]:
import numpy as np
import pandas as pd

In [172]:
dataset = pd.read_csv("../data/training_data.csv")

In [173]:
dataset.columns = dataset.columns.str.strip()  # Removes leading and trailing spaces
print(dataset.isnull().sum()) # Check for missing values
print("---")
print(dataset.duplicated().sum()) # Check for duplicate values


step              0
type              0
amount            0
nameOrig          1
oldbalanceOrg     1
newbalanceOrig    1
nameDest          1
oldbalanceDest    1
newbalanceDest    1
isFraud           1
isFlaggedFraud    1
dtype: int64
---
0


In [174]:
dataset.dropna(inplace=True)
dataset.drop_duplicates(inplace=True)

In [175]:
dataset = dataset[[ 'amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest','isFraud']] 
dataset

,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,386385.08,4669568.85,5055953.92,506035.06,119649.98,0.0
1,212458.78,234635.00,447093.78,806037.88,593579.10,0.0
2,19967.60,3634.00,0.00,0.00,0.00,0.0
3,527616.51,180216.00,0.00,92157.10,619773.61,0.0
4,206067.85,0.00,0.00,2131494.48,2337562.32,0.0
...,...,...,...,...,...,...
143920,37377.47,26397.78,0.00,753020.04,790397.50,0.0
143921,184782.46,1512205.59,1696988.05,293982.59,109200.13,0.0
143922,31391.33,31152.00,0.00,0.00,0.00,0.0
143923,255521.98,1237508.67,1493030.65,768396.97,512874.99,0.0


In [176]:
'''
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Assuming your dataset is already loaded, e.g.:
# dataset = pd.read_csv("your_file.csv")

# Select independent features and dependent target
X = dataset[['amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest']] # Features
Y = dataset['isFraud'] # Target

# Create a new DataFrame that includes both X and Y for plotting
data_with_target = X.copy()  # Copy X to avoid modifying it directly
data_with_target['isFraud'] = Y  # Add the target 'isFraud' to the DataFrame

# Plot pairwise scatter plots for each feature in X against Y
for col in X.columns:
    plt.figure(figsize=(6, 4))
    sns.scatterplot(x=col, y='isFraud', data=data_with_target)
    plt.title(f'Scatter plot of {col} vs isFraud')
    plt.show()
'''

'\nimport seaborn as sns\nimport matplotlib.pyplot as plt\nimport pandas as pd\n\n# Assuming your dataset is already loaded, e.g.:\n# dataset = pd.read_csv("your_file.csv")\n\n# Select independent features and dependent target\nX = dataset[[\'amount\',\'oldbalanceOrg\',\'newbalanceOrig\',\'oldbalanceDest\',\'newbalanceDest\']] # Features\nY = dataset[\'isFraud\'] # Target\n\n# Create a new DataFrame that includes both X and Y for plotting\ndata_with_target = X.copy()  # Copy X to avoid modifying it directly\ndata_with_target[\'isFraud\'] = Y  # Add the target \'isFraud\' to the DataFrame\n\n# Plot pairwise scatter plots for each feature in X against Y\nfor col in X.columns:\n    plt.figure(figsize=(6, 4))\n    sns.scatterplot(x=col, y=\'isFraud\', data=data_with_target)\n    plt.title(f\'Scatter plot of {col} vs isFraud\')\n    plt.show()\n'

In [177]:
# Find column indices and all potential splits at said indies
def potential_splits(data):
    pot_splits = {}
    n_col = data.shape[1]
    
    for i in range(n_col - 1):  # Exclude the last column (target column)
        col_name = data.columns[i]  # Get the column name
        pot_splits[col_name] = []   # Use the column name instead of index
        
        # Use .iloc to access column values by index
        val = data.iloc[:, i]
        uni_val = np.unique(val)
        
        for j in range(len(uni_val)):
            split = uni_val[j]
            pot_splits[col_name].append(split)  # Store split values under column name
    
    return pot_splits

potential_splits(dataset)

{'amount': [np.float64(0.0),
  np.float64(0.03),
  np.float64(0.87),
  np.float64(1.11),
  np.float64(1.12),
  np.float64(1.38),
  np.float64(1.76),
  np.float64(2.34),
  np.float64(2.45),
  np.float64(2.48),
  np.float64(3.22),
  np.float64(3.27),
  np.float64(3.39),
  np.float64(3.41),
  np.float64(3.59),
  np.float64(3.65),
  np.float64(3.91),
  np.float64(4.33),
  np.float64(4.44),
  np.float64(4.66),
  np.float64(4.82),
  np.float64(4.94),
  np.float64(4.95),
  np.float64(4.97),
  np.float64(5.09),
  np.float64(5.16),
  np.float64(5.28),
  np.float64(5.4),
  np.float64(5.75),
  np.float64(6.22),
  np.float64(6.57),
  np.float64(6.74),
  np.float64(6.83),
  np.float64(7.01),
  np.float64(7.08),
  np.float64(7.47),
  np.float64(7.71),
  np.float64(7.84),
  np.float64(8.12),
  np.float64(8.29),
  np.float64(8.61),
  np.float64(8.65),
  np.float64(8.86),
  np.float64(8.9),
  np.float64(9.28),
  np.float64(9.55),
  np.float64(9.62),
  np.float64(9.72),
  np.float64(9.95),
  np.float64(

In [178]:
# Function to split data based on column name and value
def split_data(data, split_col, split_val):
    left_data = data[data[split_col] <= split_val]
    right_data = data[data[split_col] > split_val]
    
    return left_data, right_data

split_data(dataset,'oldbalanceDest',50000)

(          amount  oldbalanceOrg  newbalanceOrig  oldbalanceDest  \
 2       19967.60        3634.00            0.00             0.0   
 5         141.42         174.00           32.58             0.0   
 7        5292.90         257.00            0.00             0.0   
 11      39570.59       30591.00            0.00             0.0   
 14       4570.35       16060.85        11490.50             0.0   
 ...          ...            ...             ...             ...   
 143912   4404.25       46144.00        41739.75             0.0   
 143914   8270.86       40270.00        31999.14             0.0   
 143915    569.04      184929.21       184360.18             0.0   
 143919  12651.80           0.00            0.00             0.0   
 143922  31391.33       31152.00            0.00             0.0   
 
         newbalanceDest  isFraud  
 2                 0.00      0.0  
 5                 0.00      0.0  
 7                 0.00      0.0  
 11            39570.59      0.0  
 14    

In [179]:
# Subsetting dataset
s_dict = dataset['isFraud'].value_counts().to_dict()
for k,v in s_dict.items():
    print(k,"-->",v)

subset= dataset[dataset['isFraud']==1.0]
subset.head()

0.0 --> 143733
1.0 --> 192


,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
550,1735466.52,1735466.52,0.00,0.00,0.00,1.0
636,316877.85,316877.85,0.00,0.00,0.00,1.0
864,78810.30,78810.30,0.00,802788.52,881598.82,1.0
870,513082.74,513082.74,0.00,32452.79,545535.53,1.0
1472,2542664.27,2542664.27,2542664.27,0.00,0.00,1.0


In [180]:
# Get probabilities faster
target = dataset[dataset.columns[-1]]
counts = []
p_dict = target.value_counts().to_dict()
for k,v in p_dict.items():
    counts.append(v)
counts = np.array(counts) # Allows element-wise operations
print(counts.sum())

143925


In [181]:
# NEW calculate entropy 

def entropy(data): # Takes dataset instead of column
    target_col = data[data.columns[-1]]  # Makes last column of dataset the target column
    counts = []
    target_dict = target_col.value_counts().to_dict()
    for k,v in target_dict.items():
        counts.append(v)
    counts = np.array(counts)

    p = counts / counts.sum() # Does calculation at once, so faster
    ent = -np.sum(p * np.log2(p))

    return ent

entropy(dataset)

np.float64(0.014663273789461197)

In [182]:
# NEW calculate entropy of children

def total_entropy(left_child, right_child):
    total = len(left_child) + len(right_child)

    p_left = len(left_child)/total
    p_right = len(right_child)/total

    ent_tot = (p_left * entropy(left_child)) + (p_right * entropy(right_child)) # Can use asame ent func. this since entropy now takes subset of dataset

    return ent_tot

total_entropy(dataset,dataset)

np.float64(0.014663273789461197)

In [ ]:
# Get best column and attribute in that column to split at
def get_best_split(data, pot_splits):
    best_ent = np.inf  # Initialize entropy
    best_col = None    # Initialize best column
    best_val = None    # Initialize best value

    for k in pot_splits:
        for v in pot_splits[k]:
            left_child, right_child = split_data(data, k, v)
            current_ent = total_entropy(left_child, right_child)

            if current_ent < best_ent:
                best_ent = current_ent
                best_col = k
                best_val = v

    return best_col, best_val

poten_splits = potential_splits(dataset)
get_best_split(dataset,poten_splits)

In [184]:
np.unique(dataset[dataset.columns[-1]])


array([0., 1.])

In [ ]:
# Classify impure node

def classify_data(data):
  target_col = data[:,- 1]
  uni_class, count_uni = np.unique(target_col, return_counts=True)

  index = count_uni.argmax()
  classification = uni_class[index]

  return classification

In [185]:
def check_pure(data):
    if len(np.unique(data[data.columns[-1]]))==1:
        return True
    else:
        return False

check_pure(dataset)

False

In [ ]:
def decision_tree(data, min_samples, max_depth, depth=0):

    # Check for base cases
    if (check_pure(data) == True) or (len(data) <= min_samples) or (depth >= max_depth):
        if type(data) == pd.DataFrame:
          data =data.values
        classification = classify_data(data)
        return classification  # Get the majority class in case of a tie

    else:
        depth += 1
        # Determining the child nodes
        pot_splits = potential_splits(data)
        split_col, split_val = get_best_split(data, pot_splits)
        left_child, right_child = split_data(data, split_col, split_val)

        # Making the tree
        cond = f"{split_col} <= {split_val}"
        sub_tree = {cond: []}  # Tree is in the form of a dictionary

        # Recursively build child nodes
        node_yes = decision_tree(left_child, min_samples, max_depth, depth)
        node_no = decision_tree(right_child, min_samples, max_depth, depth)

        # Append children to the current node
        sub_tree[cond].append(node_yes)
        sub_tree[cond].append(node_no)

        return sub_tree

tree=decision_tree(dataset,100,1)
    
    

In [187]:
example = dataset.iloc[0]
example

amount             386385.08
oldbalanceOrg     4669568.85
newbalanceOrig    5055953.92
oldbalanceDest     506035.06
newbalanceDest     119649.98
isFraud                 0.00
Name: 0, dtype: float64

In [ ]:
# Classify from an example
example = dataset.iloc[1]

def classify_example(example,tree):
  cond = list(tree.keys())[0]
  feature, compare, val = cond.split()

  # Split condition
  if example[feature] <= float(val):
    answer = tree[cond][0]
  else:
    answer = tree[cond][1]

  # See if label reached
  if type(answer) != dict:
   return answer
  else:
   return classify_example(example,answer) # Answer is the remaining part of the tree

In [190]:
# Accuracy of classifictions

def accuracy(data, tree):
    data["classification"] = data.apply(classify_example, axis=1, args=(tree,)) # Classify each row using the decision tree
    data["correctness"] = data["classification"] == data['isFraud'] # Compare classification with the actual labels
    
    accuracy_score = data["correctness"].mean() # Calculate accuracy as the mean of correctness

    return accuracy_score



In [ ]:
# Entropy calculation for an attribute in a column
'''
def entropy_attribute(col,val,target):
    subset = dataset[dataset[col]==val] # Make subset based on attribute 
    
    ent = 0 # Initialize entropy value

    target_dict = subset[target].value_counts().to_dict()
    for k,v in target_dict.items():
        p = v/len(subset) # Occurences of 1s/0s divided by len of column
        ent -= p * np.log2(p) # Entropy of attribute

    return ent

entropy_attribute('isFraud',1.0,'isFraud')
'''

np.float64(0.0)

In [ ]:
# Entropy of parent
'''
def entropy_dataset(target):
    target_dict = dataset[target].value_counts().to_dict()
    ent = 0
    for k,v in target_dict.items():
        p = v/len(dataset)
        ent -= p * np.log2(p)
    
    return ent

entropy_dataset('isFraud')
'''

np.float64(0.014663273789461197)

In [ ]:
# Information Gain Calculation
'''
def information_gain(col,target):
    len_data = len(dataset) # Length of dataset
    att_dict = dataset[col].value_counts().to_dict() # Make dictionary of unique attribute and no. of occurences

    ent_data = entropy_dataset(target) # Calculate entropy of dataset
    ent_att = 0 # Initalize entropy of attributes of a column

    for k,v in att_dict.items():
        p = v / len_data # No. of occurences of attributes / length of dataset
        ent_att -= p * entropy_attribute(col,k,target) # Sum of entropy of all attributes in a column
    
    info_gain = ent_data + ent_att # Calculate information gain

    return info_gain

information_gain('type','isFraud')
'''

"\ndef information_gain(col,target):\n    len_data = len(dataset) # Length of dataset\n    att_dict = dataset[col].value_counts().to_dict() # Make dictionary of unique attribute and no. of occurences\n\n    ent_data = entropy_dataset(target) # Calculate entropy of dataset\n    ent_att = 0 # Initalize entropy of attributes of a column\n\n    for k,v in att_dict.items():\n        p = v / len_data # No. of occurences of attributes / length of dataset\n        ent_att -= p * entropy_attribute(col,k,target) # Sum of entropy of all attributes in a column\n    \n    info_gain = ent_data + ent_att # Calculate information gain\n\n    return info_gain\n\ninformation_gain('type','isFraud')\n"